In [1]:
import glob
import os
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import tensorflow as tf    # 대망의 텐서플로우!
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해

# データを持って来てモデル訓練できるように変換する

In [2]:


txt_file_path = 'lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['"Don\'t worry about a thing,', "'Cause every little thing gonna be all right.", 'Singin\': "Don\'t worry about a thing,']


In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


txtを一列ごとに読んで必要な単語だけセーブ

In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> don t worry about a thing , <end>',
 '<start> cause every little thing gonna be all right . <end>',
 '<start> singin don t worry about a thing , <end>',
 '<start> cause every little thing gonna be all right ! rise up this mornin , <end>',
 '<start> smiled with the risin sun , <end>',
 '<start> three little birds <end>',
 '<start> perch by my doorstep <end>',
 '<start> singin sweet songs <end>',
 '<start> of melodies pure and true , <end>',
 '<start> sayin , this is my message to you ou ou singin don t worry bout a thing , <end>']

文章の内容がある場合のみセーブ

In [5]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters='',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.
    
    
    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.
    
    # tokenの数が１５個超過は捨てる
    temp = []
    for item in tensor:
        if len(item) > 15: continue
        temp.append(item)
        
        
    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(temp, padding='post') 
    
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[    2    37    15 ...     0     0     0]
 [    2    67   133 ...     0     0     0]
 [    2  1541    37 ...     0     0     0]
 ...
 [    2     5    22 ...     0     0     0]
 [    2    24     6 ...     0     0     0]
 [    2    67 10154 ...     0     0     0]] <keras_preprocessing.text.Tokenizer object at 0x7f8bd6186250>


持ってきた歌詞たちをtoken化する   
12000個の単語のみ、一つの文章が１５個を超えたら捨てる   
そしてvetor化する

In [6]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[  2  37  15 717 113   9 180   4   3   0   0   0   0   0]
[ 37  15 717 113   9 180   4   3   0   0   0   0   0   0]


これをする理由は   
src_inputのあとはどんな単語が出てばいいのかのため   
   
src_input[0]が出た場合はtgt_input[0]が出てほしい事

In [7]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                                tgt_input, 
                                                                test_size=0.2,
                                                                shuffle=True)

print("clear")

clear


In [8]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124981, 14)
Target Train: (124981, 14)


# モデル作成

In [28]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

BUFFER_SIZEこれは多分c言語なのですることだと思います   
source,targetデータをtf.data.Datasetに入れて使います

In [29]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

print("clear")

clear


これは一体なんですかと思ったんですが   
多分こういうやってモデル作成できますよとの事を教えてくれるだと思います   
   
embedding_size, hidden_sizeはデータ数が多ければこれも多くなるらしいです   
そしてデータ数が少ない場合これを多くしたら良くないそうです

In [30]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)


<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-8.3784653e-05, -9.3008959e-05, -1.9470202e-04, ...,
          3.2312921e-04,  3.9776723e-04,  1.8253100e-04],
        [-1.7840360e-04,  1.6349106e-04, -2.5762719e-04, ...,
          6.4840657e-04,  7.5954106e-04,  5.2867682e-07],
        [-1.6594067e-04,  6.6412892e-04, -8.6580032e-05, ...,
          8.3341182e-04,  8.4973319e-04,  3.5475849e-04],
        ...,
        [-1.4341930e-03, -3.9288329e-04,  4.3908431e-04, ...,
          3.5816722e-04,  1.3520631e-03, -4.5176566e-04],
        [-1.8607904e-03, -9.0951764e-04,  5.0287176e-04, ...,
          3.7961412e-04,  1.4407263e-03, -3.4966820e-04],
        [-2.2876437e-03, -1.3912711e-03,  5.5235997e-04, ...,
          3.8934551e-04,  1.5073174e-03, -2.7405837e-04]],

       [[-8.3784653e-05, -9.3008959e-05, -1.9470202e-04, ...,
          3.2312921e-04,  3.9776723e-04,  1.8253100e-04],
        [ 7.8392870e-05, -8.9513400e-05, -1.2877352e-04, ...,
          7.0505455e-04, 

take(1)これをする理由は   
https://stackoverflow.com/questions/55435784/what-does-example-mnist-train-take1   
tupleデータを得るためらしいです

In [31]:
model.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_2 (LSTM)                multiple                  5246976   
_________________________________________________________________
lstm_3 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_1 (Dense)              multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


# モデル訓練

In [32]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)


# the save point
checkpoint_dir = 'lyricist/models/'

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_dir,
                                                 save_weights_only=True,
                                                 monitor='loss',
                                                 mode='auto',
                                                 save_best_only=True,
                                                 verbose=1)
print("✅")

model.compile(loss=loss, optimizer=optimizer)

✅


In [13]:
model.fit(dataset,
          epochs=30,
         callbacks=[cp_callback])

✅
Epoch 1/30
488/488 [==============================] - ETA: 0s - loss: 3.4889
Epoch 00001: loss improved from inf to 3.48887, saving model to lyricist/models/
488/488 [==============================] - 66s 136ms/step - loss: 3.4889
Epoch 2/30
488/488 [==============================] - ETA: 0s - loss: 3.0304
Epoch 00002: loss improved from 3.48887 to 3.03039, saving model to lyricist/models/
488/488 [==============================] - 66s 135ms/step - loss: 3.0304
Epoch 3/30
488/488 [==============================] - ETA: 0s - loss: 2.8658
Epoch 00003: loss improved from 3.03039 to 2.86578, saving model to lyricist/models/
488/488 [==============================] - 65s 133ms/step - loss: 2.8658
Epoch 4/30
488/488 [==============================] - ETA: 0s - loss: 2.7419
Epoch 00004: loss improved from 2.86578 to 2.74191, saving model to lyricist/models/
488/488 [==============================] - 65s 133ms/step - loss: 2.7419
Epoch 5/30
488/488 [==============================] - ETA: 0s 

そして訓練します   
embedding_size, hidden_size数値を変えてみたかったんですが   
これだけでも時間がかかりすぎて勇気が出なかったんです   
   
前回学んだモデルセーブ機能を使ってみたり

In [14]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
																 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.


print("clear")

clear


入力した文字を基に次の文字を探して出します   
次の文字がないとかmax_len=20を超えた場合returnして文章を出してくれます

# どれだけ予測できるのかを見ます

In [41]:
dataset2 = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
dataset2 = dataset2.batch(BATCH_SIZE, drop_remainder=True)

In [42]:
results = model.evaluate(dataset)
print("✅")

488/488 [==============================] - 23s 48ms/step - loss: 1.1581
✅


In [46]:
# loss
print("loss value: {:.3f}".format(results))

loss value: 1.158


# 実際に使ってみる

In [24]:
generate_text(model, tokenizer, init_sentence="<start> i am")

'<start> i am the one who loves <end> '

# ここからは私なりのテスト

In [33]:
model.load_weights(checkpoint_dir)

セーブしたモデルを持ってくる

In [36]:
generate_text(model, tokenizer, init_sentence="<start> i am")

'<start> i am the one who loves <end> '

良くできる